In [6]:
from bs4 import BeautifulSoup
import urllib.request

In [9]:
fp = urllib.request.urlopen("http://www.elections.ny.gov/ENR/NYSENRAccessible.html")
mybytes = fp.read()

mystr = mybytes.decode("utf8")
fp.close()

In [10]:
len(mystr)

962853

In [11]:
soup = BeautifulSoup(mystr, 'html.parser')

In [21]:
print(soup.prettify())

﻿
<!DOCTYPE HTML>
<!-- saved from url=(0038)http://localhost:58378/ENRResult/Index -->
<!DOCTYPE html PUBLIC "" "">
<html lang="en">
 <head>
  <meta content="IE=11.0000" http-equiv="X-UA-Compatible"/>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <title>
   NY Election Night Results
  </title>
  <link href="NYSENRAccessible_files/bootstrap.css" rel="stylesheet"/>
  <link href="NYSENRAccessible_files/site.css" rel="stylesheet"/>
  <script src="NYSENRAccessible_files/modernizr-2.6.2.js">
  </script>
  <script type="text/javascript">
   var _gaq = _gaq || [];
            _gaq.push(['_setAccount', 'UA-4287801-4']);
            _gaq.push(['_trackPageview']);

            (function () {
                var ga = document.createElement('script'); ga.type = 'text/javascript'; ga.async = true;
                ga.src = ('https:' == document.location.protocol ? 'https://ssl' : 'http://www') + '.google-analytics.com/ga.js';
                var

In [71]:
info = [{
    "h4": h.text,
    "p": h.find_next("p").text,
    "table": h.find_next("table").text,
    "rows": [h.find_next("table").find_all("tr")]} for h in soup.find_all("h4")]

In [81]:
info[1]["rows"][0].find_all("th")

[<th id="c1" scope="col">Candidate Name</th>,
 <th id="c2" scope="col">Party</th>,
 <th id="c3" scope="col">Total Votes</th>,
 <th id="c4" scope="col">Votes Percentage</th>]